The primary purpose of this notebook is to compute the FIM for each configuration with respect to the environmental parameters.
A configuration is defined by the source range and receiver depth.
This is equivalent to specifying the location (range and depth) of the receiver from the source.

**Disclaimer:** This calculation requires `uwlib` and `fimpack`, which are proprietary modules developed at BYU.

I will try to make this notebook accessible for those who don't have access to these packages, by exporting what I calculated and include them in this repo.
However, the calculation to generate the FIM data will also be included here.

In [ ]:
from pathlib import Path
import os
import tarfile
import itertools

import numpy as np

# import uwlib
# from fimpack import (
#     orca_manager,
#     param_conditioning,
#     matrix_manager,
#     derivative_analysis,
# )
from information_matching.utils import set_directory

In [ ]:
# Setting up directories
WORK_DIR = Path().absolute()
DATA_DIR = WORK_DIR / "data"
SVP_DIR = DATA_DIR / "svp"

In [ ]:
# To help those who don't have access to uwlib and fimpack, let's try to get the
# FIM with respect to the environmental parameters from the tarball that I generated.
FIM_DIR = DATA_DIR / "FIMs" / "environment"
tarball = FIM_DIR.parent / "fim_environment.tar.gz"

if not FIM_DIR.exists():
    # Check if tarball exists. If it does, then extract.
    if tarball.exists():
        os.chdir(FIM_DIR.parent)  # Entering the FIM directory
        # Extracting the tarball
        with tarfile.open(tarball.name) as f:
            f.extractall()
        os.chdir(WORK_DIR)  # Exiting FIM directory, go back to the previous path
    else:
        # There is no FIM data. This is mainly for me to generate the data.
        FIM_DIR.mkdir()

In [ ]:
# List of TOML files
opt_toml_file = DATA_DIR / "FIM_opt.toml"
# Sound profile files
sediment_type_list = ["mud", "clay", "silt", "sand", "gravel"]
svp_toml_path_dict = {
    sed: SVP_DIR / f"svp_{sed}_35m_unit_test.toml" for sed in sediment_type_list
}
# List of sound source frequencies
freq_list = [50, 100, 200, 400]  # In Hz

The receiver depth values are the same as what's used in Michael's paper.
The source (or receiver) range is set up equally spaced from 1 km to 5 km, every 200 m.
Not that I just chose these numbers arbitrarily.
In Michael's paper, he only used 1 source range value, that is 3 km.

In [ ]:
# Source depth, receiver depth, and receiver range
source_range = np.linspace(1, 5, 21)
source_depth = np.array([8, 16])
receiver_depth = np.arange(5, 76, 5)

# Compute FIM for each configuration

In [ ]:
configs = list(itertools.product(source_range, receiver_depth))
print("Number of configurations:", len(configs))

In [ ]:
for sediment_type in sediment_type_list:
    # Iterate over the sediment layer types
    svp_toml_file = svp_toml_path_dict[sediment_type]
    print("Sediment:", sediment_type)

    FIM_SED_DIR = set_directory(FIM_DIR / sediment_type)

    for freq in freq_list:
        # Iterate over the frequencies
        print("Frequency:", freq, "Hz")
        
        # There is one case that gave me issue, which is gravel at 100 Hz. So, we will
        # skip FIM calculation for this case.
        if sediment_type=='gravel' and freq==100:
            continue

        FIM_SED_FREQ_DIR = set_directory(FIM_SED_DIR / f"f{int(freq)}Hz")
        for ii, conf in enumerate(configs):
            # Iterate over configuration
            sr, rd = conf
            rd = [rd]
            print(f"{ii}: Source range: {sr:0.1f} km, Receiver depth: {int(rd[0])} m")

            fim_file_path = FIM_SED_FREQ_DIR / f"config_{ii}.npz"
            if not fim_file_path.exists():
                # Initialize ORCA
                orca = orca_manager.initialize_orca(
                    freq, str(svp_toml_file), str(opt_toml_file)
                )

                # Compute derivative of transmission loss wrt Green's function
                TL, n_modes, gf = uwlib.tl.calc_tl_from_orca(
                    orca, freq, sr, source_depth, rd, return_gf=True, return_nmode=True
                )
                gf = np.abs(gf)
                gf = np.squeeze(gf.flatten())
                J_TL = np.diag(20 * np.log10(np.exp(1)) / gf)

                # Parameter preconditioning
                x_dict, num_layers = orca_manager.get_x_dict(orca)
                scale_dict = param_conditioning.get_x_to_theta_scale_dict(
                    orca, x_dict, freq, sr
                )
                theta_dict = param_conditioning.scale_x_to_theta(
                    x_dict, scale_dict, frequency=freq
                )
                # Apparently, we can only use 1 source range value for this preconditioning. If we have
                # multiple source range values, I guess we should loop over each value.
                active_labels_list, theta_vector = param_conditioning.get_active_labels(
                    theta_dict
                )
                theta_labels = active_labels_list
                (
                    phi,
                    phi_flags,
                    x_hold_constant_values,
                ) = param_conditioning.theta_to_phi(
                    active_labels_list, custom_parameterization="n"
                )
                # Derivative of phi wrt theta
                dphi_dtheta = matrix_manager.calculate_dphi_dtheta(
                    theta_vector, theta_labels, phi
                )

                # Initialize derivative result storage
                result_storage = derivative_analysis.Deriv_Results(freq, sediment_type)

                # Derivative of Green's function wrt phi
                jacobian_theta = matrix_manager.construct_jacobian(
                    theta_vector,
                    theta_labels,
                    orca,
                    scale_dict,
                    phi_flags,
                    dphi_dtheta,
                    freq,
                    sr,
                    source_depth,
                    rd,
                    num_layers,
                    result_storage,
                )

                # Derivative of the transmission loss wrt theta
                # Note: theta in the function name is phi in the paper and x in the
                # function name is theta in the paper.
                jacobian = matrix_manager.J_theta_to_J_x(
                    jacobian_theta, J_TL, theta_labels, scale_dict, freq
                )
                # Note: This transformation already assume that the derivative is taken
                # wrt log parameters
                fim = jacobian.T @ jacobian

                # Export
                np.savez(fim_file_path, jacobian=jacobian, fim=fim)

In [ ]:
# # Create a tarball
# if not tarball.exists():
#     os.chdir(FIM_DIR.parent)  # Entering the FIM directory
#     # Creating a tarball
#     with tarfile.open(tarball.name, "w:gz") as tar:
#         tar.add(FIM_DIR.name)
#     os.chdir(WORK_DIR)  # Exiting the FIM directory, go back to the previous path